# Python script for data transformation 

## BRICARE:

BRICARE consists of 2 different types of files by year:

a. File after 2022 (2023-2024) = 79 kolom


b. File before 2022 (2019-2022) = 27 kolom


### File Type A


Data Extraction for File Type A must be 2 Files:


A.1 Columns (without "Details")


A.2 Details only 

Columns to be cleansed or Transform:
- All columns with values "None", "NaN, "N/A", "NULL"
- These columns must follow this datetime format: format='%Y-%m-%d %H:%M:%S' or format='%Y-%m-%d %H:%M:%S.%f' 

['Create_Date','TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']

- Remove all unknown characters e.g. \ufeff in column "Ticket_ID"

- Columns shoud be mapped based on their Call_Type_ID:

['Produk','Jenis_Produk','Jenis_Laporan']

- PLEASE ADD CIF

#### File A.1 Columns (without "Details"). Please use this if the file is txt

In [45]:
import pandas as pd
import re
import numpy as np

# 78 Columns
column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

def parse_file(file_path):

    data = []
    date_pattern = re.compile(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3}')

    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            parts = line.strip().split(';')

            date_index = next(i for i, part in enumerate(parts) if date_pattern.match(part))

            ticket_id = parts[0] 
            call_type_id = parts[1]  
            description = ';'.join(parts[2:date_index])  
            create_date = parts[date_index]  

      
            data.append([ticket_id, call_type_id, description, create_date] + parts[date_index + 1:])


    df = pd.DataFrame(data, columns=column_names)

    df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')

    return df


file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_1masking.txt"

df = parse_file(file_path)
df.replace('NULL', np.nan, inplace=True)
df.replace('None', np.nan, inplace=True)
df.replace('N/A', np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')


columns_to_convert = ['TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']
for column in columns_to_convert:
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)
   

df['Ticket_ID'] = df['Ticket_ID'].apply(lambda x: x.replace('\ufeff', '').strip())



df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')
startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"


df.to_csv(filename, index=False)

print(f"Data saved to {filename}")

Data saved to bricare_20230101_20230101.csv


C:\Users\maste\AppData\Local\Temp\ipykernel_51572\1840199040.py:51: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('NULL', np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_51572\1840199040.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('None', np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_51572\1840199040.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with

#### File A.1 Columns (without "Details"). Please use this if the file is csv

In [46]:
import pandas as pd
import numpy as np

# 78 Columns
column_names = [
    "Ticket_ID", "Call_Type_ID", "Call_Type", "Create_Date", "gateway", "Jenis_Laporan", "Nama_Nasabah", 
    "No_Rekening", "Nominal", "status", "TanggalClosed", "tanggalTransaksi", "Chanel", "Fitur", "Nomor_Kartu", 
    "user_group", "assgined_to", "attachment_done", "email", "full_name", "no_telepon", "approver_login", 
    "approver_name", "SLAResolution", "submitter_login_id", "submitter_user_group", "user_login_name", 
    "Jenis_Produk", "Last_Modified_By", "Merchant_ID", "Modified_Date", "NOTAS", "Produk", "SLA_Status", "TID", 
    "tanggalAttachmentDone", "Tgl_Assigned", "Tgl_Eskalasi", "AnalisaSkils", "Attachment_", "Bank_BRI", 
    "Biaya_Admin", "Suku_Bunga", "Bunga", "Butuh_Attachment", "Cicilan", "Hasil_Kunjungan", "Log_Name", 
    "MMS_Ticket_Id", "Mass_Ticket_Upload_Flag", "Nama_Supervisor", "Nama_TL", "Nama_Wakabag", "Nasabah_Prioritas", 
    "Notify_By", "Organization", "Output_Settlement", "phone_survey", "Return_Ticket", "Settlement_By", 
    "Settlement_ID", "Settlement", "Site_User", "Status_Return", "Status_Transaksi", "Submitter_Region", 
    "Submitter_SiteGroup", "Submitter_User_group_ID", "Tanggal_Settlement", "Tgl_Foward", "Tgl_In_Progress", 
    "Tgl_Returned", "Ticket_Referensi", "Tiket_Urgency", "Tipe_Remark", "UniqueID", "users", "Usergroup_ID"
]

def parse_csv(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            parts = line.strip().split(';')
            if len(parts) > 78:
                description = ';'.join(parts[2:-75])
                new_parts = parts[:2] + [description] + parts[-75:]
                data.append(new_parts)
            else:
                data.append(parts)
    
    df = pd.DataFrame(data, columns=column_names)
    df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')
    
    df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
    df.fillna('', inplace=True)
    df = df.replace(['0', 0], '')

    columns_to_convert = ['TanggalClosed', 'tanggalTransaksi','Modified_Date','tanggalAttachmentDone','Tgl_Assigned','Tgl_Eskalasi','Tanggal_Settlement','Tgl_Foward','Tgl_In_Progress','Tgl_Returned']
    for column in columns_to_convert:
        df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)
    
    df['Ticket_ID'] = df['Ticket_ID'].apply(lambda x: x.replace('\ufeff', '').strip())

    return df
file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_1masking.csv"
df = parse_csv(file_path)

startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"
df.to_csv(filename, index=False)

filename


C:\Users\maste\AppData\Local\Temp\ipykernel_51572\2385745217.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
C:\Users\maste\AppData\Local\Temp\ipykernel_51572\2385745217.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


'bricare_20230101_20230101.csv'

#### Cleasing the master Call Type file

In [34]:
import pandas as pd

master_df_path = r"C:\Users\maste\Downloads\bricare\(REVISED) SLA-OLA_NewUserGrouping_Ringkasan Kirim ME Versi 1.6.csv"
df = pd.read_csv(master_df_path, sep=';')


df.replace('NULL', np.nan, inplace=True)
df.replace('None', np.nan, inplace=True)
df.replace('N/A', np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')
df = df.dropna(how='all')
df.iloc[:450]
df.to_csv("master_calltype.csv", index=False)

C:\Users\maste\AppData\Local\Temp\ipykernel_24036\2288781540.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


#### Call type mapping for columns 'Produk', 'Jenis Produk', 'Jenis Laporan'

In [47]:
import pandas as pd


user_dataset_path = r"D:\dataquality\bricare_20230101_20230101.csv"
user_df = pd.read_csv(user_dataset_path)
master_df_path = r"D:\dataquality\master_calltype.csv"
master_df = pd.read_csv(master_df_path)


master_df = master_df.rename(columns={
    'Case Types': 'Call_Type_ID', 
    'Product': 'Produk', 
    'Sub Product': 'Jenis_Produk', 
    'Case Category': 'Jenis_Laporan'
})


user_df['Call_Type_ID'] = user_df['Call_Type_ID'].astype(str)
master_df['Call_Type_ID'] = master_df['Call_Type_ID'].astype(str)


merged_df = pd.merge(user_df, master_df[['Call_Type_ID', 'Produk', 'Jenis_Produk', 'Jenis_Laporan']], on='Call_Type_ID', how='left')

user_df['Produk'] = merged_df['Produk_y']
user_df['Jenis_Produk'] = merged_df['Jenis_Produk_y']
user_df['Jenis_Laporan'] = merged_df['Jenis_Laporan_y']

user_df.to_csv(user_dataset_path, index=False)

#### File A.2 Details only. Please use this if the file is txt

In [48]:
import pandas as pd

def process_text_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove BOM from each line
    lines = [line.replace('\ufeff', '') for line in lines]

    entries = []
    current_entry = []
    current_ticket_id = None

    for line in lines:
        if line.startswith('TTB'):
            if current_entry:  
                entries.append((current_ticket_id, '\n'.join(current_entry)))
                current_entry = []
        
            parts = line.split(',', 3)
            if len(parts) > 3:
                current_ticket_id = parts[0]  
                current_entry.append(parts[3].strip())  
            continue
        current_entry.append(line.strip())

    if current_entry:
        entries.append((current_ticket_id, '\n'.join(current_entry)))

    return entries


def remove_bom_and_strip(df):
    return df.applymap(lambda x: x.replace('\ufeff', '').strip() if isinstance(x, str) else x)


file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_2_details.txt"
processed_data = process_text_data(file_path)


df_final = pd.DataFrame(processed_data, columns=['Ticket ID', 'Details'])

if df_final.iloc[0]['Ticket ID'] and df_final.iloc[0]['Details'].startswith(df_final.iloc[0]['Ticket ID']):
    df_final.at[0, 'Details'] = df_final.iloc[0]['Details'][len(df_final.iloc[0]['Ticket ID'])+2:]

df_final=df_final.iloc[:10]
df_final.iloc[:10].to_csv('details_20230101_20230101.csv', index=False)



#### File A.2 Details only. Please use this if the file is csv

In [49]:
import pandas as pd

def process_csv_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Remove BOM from each line
    lines = [line.replace('\ufeff', '') for line in lines]

    entries = []
    current_entry = []
    current_ticket_id = None

    for line in lines:
        if line.startswith('TTB'):
            if current_entry:
                entries.append((current_ticket_id, '\n'.join(current_entry)))
                current_entry = []
        
            parts = line.split(',', 3)
            if len(parts) > 3:
                current_ticket_id = parts[0]
                current_entry.append(parts[3].strip())
            continue
        current_entry.append(line.strip())

    if current_entry:
        entries.append((current_ticket_id, '\n'.join(current_entry)))

    return entries

file_path = r"C:\Users\maste\Downloads\bricare_case_januari2023_2_details.csv"
processed_data = process_csv_data(file_path)

df_final = pd.DataFrame(processed_data, columns=['Ticket ID', 'Details'])


if df_final.iloc[0]['Ticket ID'] and df_final.iloc[0]['Details'].startswith(df_final.iloc[0]['Ticket ID']):
    df_final.at[0, 'Details'] = df_final.iloc[0]['Details'][len(df_final.iloc[0]['Ticket ID'])+2:]


df_final = df_final.iloc[:10]
output_path = "details_20230101_20230101.csv"
df_final.to_csv(output_path, index=False)

print(f"Processed data saved to {output_path}")


Processed data saved to details_20230101_20230101.csv


#### Merge the file A.1 and file A.2

In [50]:
import pandas as pd

#just take 10 lines for an example
path=r"D:\dataquality\bricare_20230101_20230101.csv"
df=pd.read_csv(path)
df.iloc[:10].to_csv(path,index=False)

In [51]:
import pandas as pd


file_path_1 = r"D:\dataquality\bricare_20230101_20230101.csv"
file_path_2 = r"D:\dataquality\details_20230101_20230101.csv"


df_tenline_bricare = pd.read_csv(file_path_1)
df_detail_bricare_10line = pd.read_csv(file_path_2)

df_detail_bricare_10line.columns = ['Ticket_ID', 'Details']

merged_df = pd.merge(df_tenline_bricare, df_detail_bricare_10line, on='Ticket_ID', how='left')


output_file_path = r"D:\dataquality\bricare_20230101_20230101.csv"


column_to_move="Details"
merged_df = merged_df[[col for col in merged_df if col != column_to_move][:3] + [column_to_move] + [col for col in merged_df if col != column_to_move][3:]] 

merged_df.to_csv(output_file_path, index=False)

### File Type B


Data Extraction for File Type B (27 columns)


Columns to be cleansed or Transform:
- All columns with values "None", "NaN, "N/A", "NULL"
- These columns must follow this datetime format: format='%Y-%m-%d %H:%M:%S' or format='%Y-%m-%d %H:%M:%S.%f' 

['TanggalClosed', 'tanggalTransaksi','Create_Date']

- Remove all unknown characters e.g. \ufeff in column "Ticket_ID" if any

- PLEASE ADD CIF



In [52]:
import pandas as pd
import numpy as np

# Define the column list
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"  
]


path = r"C:\Users\maste\Downloads\BRICARE_25042024 masking.csv"


data = pd.read_csv(path, delimiter=';')


data['Column1'] = data['Column1'].astype(str)
data_cleaned = data[data['Column1'].str.match(r'TTB\d+')]

data_cleaned['Column2'] = data_cleaned['Column2'].astype(str)
data_cleaned = data_cleaned[data_cleaned['Column2'].str.match(r'^\d{4}$')]

data_cleaned['Column4'] = pd.to_datetime(data_cleaned['Column4'], errors='coerce')
data_cleaned = data_cleaned.dropna(subset=['Column4'])


data_to_drop = ['Column28', 'Column29', 'Column30', 'Column31', 'Column32']
data_cleaned = data_cleaned.drop(columns=data_to_drop)


if len(data_cleaned.columns) <= len(column_list):
    data_cleaned.columns = column_list[:len(data_cleaned.columns)]


data_cleaned.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
data_cleaned.fillna('', inplace=True)
data_cleaned = data_cleaned.replace(['0', 0], '')


columns_to_convert = ['TanggalClosed', 'tanggalTransaksi', 'Create_Date']
for column in columns_to_convert:
    data_cleaned[column] = pd.to_datetime(data_cleaned[column], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
    data_cleaned[column] = data_cleaned[column].apply(lambda x: '' if pd.isna(x) else x)

# Just take 10 lines for an example
data_cleaned = data_cleaned.iloc[:10]


data_cleaned.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')


startdate = pd.Timestamp(min(data_cleaned['Create_Date']))
enddate = pd.Timestamp(max(data_cleaned['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricare_{formatted_startdate}_{formatted_enddate}.csv"


data_cleaned.to_csv(filename, index=False)

print(f"Data saved to {filename}")


C:\Users\maste\AppData\Local\Temp\ipykernel_33500\3452762791.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['Column2'] = data_cleaned['Column2'].astype(str)


Data saved to bricare_20200101_20200101.csv


## Check the number of columns

In [55]:

path1=r"D:\dataquality\bricare_20230101_20230101.csv"
path2=r"D:\dataquality\bricare_20200101_20200101.csv"

df1=pd.read_csv(path1)
df2=pd.read_csv(path2)
print(len(df1.columns))
print(len(df2.columns))

print(df1.columns)
print(df2.columns)

79
27
Index(['Ticket_ID', 'Call_Type_ID', 'Call_Type', 'Details', 'Create_Date',
       'gateway', 'Jenis_Laporan', 'Nama_Nasabah', 'No_Rekening', 'Nominal',
       'status', 'TanggalClosed', 'tanggalTransaksi', 'Chanel', 'Fitur',
       'Nomor_Kartu', 'user_group', 'assgined_to', 'attachment_done', 'email',
       'full_name', 'no_telepon', 'approver_login', 'approver_name',
       'SLAResolution', 'submitter_login_id', 'submitter_user_group',
       'user_login_name', 'Jenis_Produk', 'Last_Modified_By', 'Merchant_ID',
       'Modified_Date', 'NOTAS', 'Produk', 'SLA_Status', 'TID',
       'tanggalAttachmentDone', 'Tgl_Assigned', 'Tgl_Eskalasi', 'AnalisaSkils',
       'Attachment_', 'Bank_BRI', 'Biaya_Admin', 'Suku_Bunga', 'Bunga',
       'Butuh_Attachment', 'Cicilan', 'Hasil_Kunjungan', 'Log_Name',
       'MMS_Ticket_Id', 'Mass_Ticket_Upload_Flag', 'Nama_Supervisor',
       'Nama_TL', 'Nama_Wakabag', 'Nasabah_Prioritas', 'Notify_By',
       'Organization', 'Output_Settlement', 'phone_

## To Compare two files

In [18]:
# Load the newly uploaded files for detailed comparison
processed_file_path_newest = r"D:\dataquality\bricare_20230101_20230101.csv"
expected_file_path_newest = r"D:\dataquality\bricare_20230101_20230101.csv"

# Read the files
processed_df_newest = pd.read_csv(processed_file_path_newest)
expected_df_newest = pd.read_csv(expected_file_path_newest)

# Check for exact match first
exact_match = processed_df_newest.equals(expected_df_newest)

# Initialize a dictionary to store detailed comparison results
comparison_details = {
    'shape_match': processed_df_newest.shape == expected_df_newest.shape,
    'columns_match': processed_df_newest.columns.equals(expected_df_newest.columns),
    'column_differences': {},
    'value_differences': {}
}

# Compare each aspect if exact match is not true
if not exact_match:
    # Check shape
    if not comparison_details['shape_match']:
        comparison_details['shape_details'] = {
            'processed_shape': processed_df_newest.shape,
            'expected_shape': expected_df_newest.shape
        }
    
    # Check columns
    if not comparison_details['columns_match']:
        comparison_details['column_details'] = {
            'processed_columns': processed_df_newest.columns.tolist(),
            'expected_columns': expected_df_newest.columns.tolist()
        }

    # Check column-by-column values
    for column in processed_df_newest.columns:
        if not processed_df_newest[column].equals(expected_df_newest[column]):
            comparison_details['column_differences'][column] = processed_df_newest[column].compare(expected_df_newest[column])

# Summarize value differences
if not comparison_details['columns_match']:
    value_differences = {}
    for column in processed_df_newest.columns:
        if not processed_df_newest[column].equals(expected_df_newest[column]):
            value_differences[column] = processed_df_newest[column].compare(expected_df_newest[column])
    comparison_details['value_differences'] = value_differences

comparison_details


{'shape_match': True,
 'columns_match': True,
 'column_differences': {},
 'value_differences': {}}

# To collect all Error logs in a path

directory_path is the error logs path as well as the location where the combined error log file will be saved


In [41]:
import os
import pandas as pd

def combine_error_logs(directory_path):
    combined_df = pd.DataFrame()

    for filename in os.listdir(directory_path):
        if filename.startswith("error"):
            file_path = os.path.join(directory_path, filename)
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Normalize the 'TICKET_ID' to ensure duplicates are identified
    combined_df['TICKET_ID'] = combined_df['TICKET_ID'].str.upper()

    combined_df = combined_df.drop_duplicates(subset='TICKET_ID')

    output_path = os.path.join(directory_path, 'error_logs.csv')
    combined_df.to_csv(output_path, index=False)
    print(f"Combined error logs saved to: {output_path}")
    return output_path, combined_df

directory_path = r"C:\Users\maste\Downloads\dataloader_v60.0.2\log"
output_path, combined_df = combine_error_logs(directory_path)


Combined error logs saved to: C:\Users\maste\Downloads\dataloader_v60.0.2\log\error_logs.csv


# To cleanse user data

In [49]:
import pandas as pd
from io import StringIO
import html


# Function to decode HTML entities in a DataFrame
def decode_html_entities(df):
    df_decoded = df.applymap(lambda x: html.unescape(x) if isinstance(x, str) else x)
    return df_decoded


file_path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\USER.txt" 

# Read the raw file content
with open(file_path, 'r') as file:
    raw_data = file.readlines()
    
# Split headers and data    
header = raw_data[0].replace("&#124;", "|").strip()
data = [line.replace("&#124;", "|").strip() for line in raw_data[1:]]

# Read the cleaned data into a pandas DataFrame
df_cleaned = pd.DataFrame([line.split('|') for line in data], columns=header.split('|'))

# Decode HTML entities
df_cleaned = decode_html_entities(df_cleaned)
df_cleaned

# Remove the quotes in Dataframe
def remove_quotes(df):
    df.columns = df.columns.str.replace('"', '')
    df = df.apply(lambda col: col.str.replace('"', '', regex=True))
    return df

df_cleaned = remove_quotes(df_cleaned)
df_cleaned
df_cleaned.to_csv("user_data.csv", index=False)

C:\Users\maste\AppData\Local\Temp\ipykernel_43760\4048578058.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_decoded = df.applymap(lambda x: html.unescape(x) if isinstance(x, str) else x)


# Working on real-like data

In [34]:
import pandas as pd

path = r"D:\Salesforce\archive\dataquality\gx\BRICARE_25042024.csv"

# Option 1: Check for quoted fields or use different delimiter
try:
    df = pd.read_csv(path, sep=',', on_bad_lines='skip') 
except pd.errors.ParserError as e:
    print(f"Error parsing file: {e}")


C:\Users\maste\AppData\Local\Temp\ipykernel_51572\3036651791.py:7: DtypeWarning: Columns (1,8,17,21,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep=',', on_bad_lines='skip')


In [32]:
import pandas as pd
import numpy as np

# Define the column list
column_list = [
    "Ticket_ID",  
    "Call_Type_ID",  
    "Call_Type", 
    "Create_Date",  
    "gateway",  
    "Jenis_Laporan",  
    "Nama_Nasabah",  
    "No_Rekening", 
    "Nominal",  
    "status",  
    "TanggalClosed", 
    "tanggalTransaksi",  
    "Chanel",  
    "Fitur",  
    "Nomor_Kartu", 
    "user_group",  
    "assgined_to",  
    "attachment_done",  
    "email",  
    "full_name",  
    "no_telepon",  
    "approver_login",  
    "approver_name",  
    "SLAResolution",  
    "submitter_login_id",  
    "submitter_user_group", 
    "user_login_name"
]

path = r"D:\Salesforce\archive\dataquality\gx\BRICARE_25042024.csv"

# Initialize lists to store data and problematic lines
data = []
problematic_lines = []

def parse_line(line):
    parts = line.split(',')
    try:
        # Identifying Ticket_ID
        ticket_id_index = next(i for i, part in enumerate(parts) if part.startswith('TTB'))
        ticket_id = parts[ticket_id_index]
        
        # Identifying Call_Type_ID
        call_type_id_index = ticket_id_index + 1
        call_type_id = parts[call_type_id_index]
        
        # Identifying Create_Date (the part that looks like a datetime string)
        create_date_index = next(i for i, part in enumerate(parts) if '-' in part and ':' in part)
        create_date = parts[create_date_index]
        
        # Call_Type is everything between Call_Type_ID and Create_Date
        call_type = ' '.join(parts[call_type_id_index + 1:create_date_index])
        
        # The rest of the fields in order
        rest = parts[create_date_index + 1:]
        
        # Combine into a single list in the correct order
        structured_line = [ticket_id, call_type_id, call_type, create_date] + rest
        
        # If the length is correct, return it
        if len(structured_line) == len(column_list):
            return structured_line
        # If there are extra fields, handle them (for example, by merging or ignoring)
        elif len(structured_line) > len(column_list):
            return structured_line[:len(column_list)]
        else:
            return None
    except StopIteration:
        # If any part of the parsing fails, consider the line problematic
        return None

# Read the file line by line and parse it
with open(path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    i = 0
    while i < len(lines):
        parsed_line = parse_line(lines[i])
        if parsed_line:
            data.append(parsed_line)
            i += 1
        else:
            # Check if merging the next line solves the issue
            if i + 1 < len(lines):
                merged_line = lines[i].strip() + ' ' + lines[i + 1].strip()
                parsed_line = parse_line(merged_line)
                if parsed_line:
                    data.append(parsed_line)
                    i += 2
                else:
                    problematic_lines.append((i, lines[i]))
                    i += 1
            else:
                problematic_lines.append((i, lines[i]))
                i += 1

# Convert the collected data into a DataFrame
df = pd.DataFrame(data, columns=column_list)


df['Ticket_ID'] = df['Ticket_ID'].astype(str)
df = df[df['Ticket_ID'].str.match(r'TTB\d+')]

df['Call_Type_ID'] = df['Call_Type_ID'].astype(str)
df = df[df['Call_Type_ID'].str.match(r'^\d{4}$')]

df['Create_Date'] = pd.to_datetime(df['Create_Date'], errors='coerce')
df = df.dropna(subset=['Create_Date'])

df.replace(['NULL', 'None', 'N/A'], np.nan, inplace=True)
df.fillna('', inplace=True)
df = df.replace(['0', 0], '')

columns_to_convert = ['TanggalClosed', 'tanggalTransaksi', 'Create_Date']
for column in columns_to_convert:
    df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
    df[column] = df[column].apply(lambda x: '' if pd.isna(x) else x)


df.drop('Unnamed: 0', axis=1, inplace=True, errors='ignore')


startdate = pd.Timestamp(min(df['Create_Date']))
enddate = pd.Timestamp(max(df['Create_Date']))

formatted_startdate = startdate.strftime('%Y%m%d')
formatted_enddate = enddate.strftime('%Y%m%d')

filename = f"bricaredatareal_{formatted_startdate}_{formatted_enddate}.csv"


df.to_csv(filename, index=False)

print(f"Data saved to {filename}")


problematic_df = pd.DataFrame(problematic_lines, columns=['Line_Number', 'Data'])


problematic_filename = f"bricaredatareal_problematic_{formatted_startdate}_{formatted_enddate}.csv"
problematic_df.to_csv(problematic_filename, index=False)

print(f"Problematic data saved to {problematic_filename}")


print(f"Number of rows in dataframe: {len(df)}")
print(f"Number of problematic lines: {len(problematic_df)}")


Data saved to bricaredatareal_20200101_20200201.csv
Problematic data saved to bricaredatareal_problematic_20200101_20200201.csv
Number of rows in dataframe: 463581
Number of problematic lines: 881


# Create masked Data for UAT


To Mask:

Nama_Nasabah
No_Rekening
full_name
no_telepon
approver_name = beda dengan Nama_Nasabah
user_login_name = beda 
Log_Name
Nama_Supervisor
Nama_TL
Nama_Wakabag

Kolom Detail:

Kode Cabang          : 0307 4 digits
No Kartu             : 5221843130736932 16 digits
No Rekening          : 030701098507501 15 digits
Nama                 : SITI SHOLEHA
No ID                : 3316022012770004

In [53]:
import pandas as pd
from faker import Faker
import random 

fake = Faker('id_ID')
pd.set_option('display.max_columns', None)


path = r"D:\dataquality\bricare_20230101_20230101.csv"
df = pd.read_csv(path)

def mask_data(df):
    df['Nama_Nasabah'] = df['Nama_Nasabah'].apply(lambda x: fake.name())
    df['No_Rekening'] = df['No_Rekening'].apply(lambda x: fake.bban())
    df['full_name'] = df['full_name'].apply(lambda x: fake.name())
    df['no_telepon'] = df['no_telepon'].apply(lambda x: fake.phone_number())
    df['approver_name'] = df['approver_name'].apply(lambda x: fake.name())
    df['user_login_name'] = df['user_login_name'].apply(lambda x: fake.user_name())
    df['Log_Name'] = df['Log_Name'].apply(lambda x: fake.user_name())
    df['Nama_Supervisor'] = df['Nama_Supervisor'].apply(lambda x: fake.name())
    df['Nama_TL'] = df['Nama_TL'].apply(lambda x: fake.name())
    df['Nama_Wakabag'] = df['Nama_Wakabag'].apply(lambda x: fake.name())
    return df


df_masked = mask_data(df)

def generate_nik():
    return f'{fake.random_number(digits=16)}'

def mask_detail(detail):
    if isinstance(detail, float) and pd.isna(detail):
        return detail
    lines = str(detail).split('\n')
    masked_lines = []
    for line in lines:
        if 'Kode Cabang' in line:
            line = f'Kode Cabang          : {fake.random_number(digits=4)}'
        elif 'No Kartu' in line:
            line = f'No Kartu             : {fake.credit_card_number()}'
        elif 'No Rekening' in line:
            line = f'No Rekening          : {fake.bban()}'
        elif 'Nama' in line:
            line = f'Nama                 : {fake.name()}'
        elif 'No ID' in line:
            line = f'No ID                : {generate_nik()}'
        masked_lines.append(line)
    return '\n'.join(masked_lines)

df_masked = df_masked.iloc[:10]


# Ensure 'Details' column is treated as string and apply mask_detail function
df_masked['Details'] = df_masked['Details'].astype(str).apply(mask_detail)



# gateway_options = [
#     'Email', 'Phone', 'Instagram', 'Walk-In', 'MMS', 
#     'Twitter', 'Facebook', 'BRImo', 'BRILink', 'Sabrina', 'Ceria'
# ]

#UAT
gateway_options = [
    'Email', 'Phone', 'Web', 'Facebook', 'Twitter'
]
# Assign random values to the 'gateway' column
df_masked['gateway'] = df_masked['gateway'].apply(lambda x: random.choice(gateway_options))




df_masked = df_masked.iloc[:10]
df_masked
output_path = 'D:\dataquality\details_4_uat.csv'
df_masked.to_csv(output_path, index=False)


# To create data to test in SIT


In [1]:
import pandas as pd


path=r"C:\Users\maste\Downloads\dataloader_v60.0.2\dummy_data_case1100000.csv"

df=pd.read_csv(path) 
df=df.iloc[:1]
df.to_csv('dummy_data_case_sit.csv', index=False)